In [1]:
from src.distance_measures import analyse_video, analyse_image
from src.gaussian_noise import generate_noise,generate_noise_avg
from src.preprocessing import process_image, process_video
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import label2rgb

In [2]:
size = 400
thresh = 0.3
disk = 3.

In [15]:
noise_avg= generate_noise_avg(size=size, threshold=thresh, disk_size=disk, amount=50)
F_,G_ = analyse_image(noise_avg,L=50)

noise= generate_noise(size=size, threshold=thresh, disk_size=disk, amount=5)
F__, G__ = analyse_video(noise, L=50)

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
im = process_video('images/cheese_gel.avi', threshold=thresh, size=size, skip_size=239)[1]
F, G = analyse_image(im, L=50)
y = F/G

In [17]:
f = np.mean(F__, axis=0)/np.mean(G__, axis=0)
loss_normal= np.sum((f - y)**2)
f_ = F_/G_
loss_acc = np.sum((f_ - y)**2)


In [18]:
print(loss_normal)
print(loss_acc)

0.06538679788533108
nan


In [ ]:
def model(thresh, disk):
    noise = generate_noise(size=size, threshold=thresh, disk_size=disk, amount=3)
    F, G = analyse_video(noise, L=50)
    return np.mean(F, axis=0)/np.mean(G, axis=0)

def loss_fn(thresh, disk):
    return np.sum((model(thresh, disk) - y)**2)

In [ ]:
max_iter = 1
#loss_list = []
step_thresh, step_disk = (0.01, 0.1)
learning_rate = 0.01

for i in range(max_iter):
    loss = loss_fn(thresh, disk)
    grad1 = (loss_fn(thresh+step_thresh, disk)-loss)/step_thresh
    grad2 = (loss_fn(thresh, disk+step_disk)-loss)/step_disk
    
    thresh -= grad1 * learning_rate
    disk -= grad2 * learning_rate

In [ ]:
print(disk)
print(thresh)

In [ ]:
noise = generate_noise(size, threshold=thresh, disk_size=disk, amount=3)
F_noise, G_noise = analyse_video(noise, L=50)
plt.plot(np.mean(F_noise, axis=0)/np.mean(G_noise, axis=0))
plt.show()

plt.imshow(label2rgb(noise[0]))
plt.show()

In [ ]:
plt.plot(F/G)
plt.show()

plt.imshow(label2rgb(im))
plt.show()